In [1]:
import pandas as pd
from datetime import date, timedelta
from tqdm import tqdm
from sodapy import Socrata

In [2]:
sdate = date(2020, 3, 28)   # start date
# sdate = date(2020, 7, 1)   # start date

edate = date.today()   # end date
delta = edate - sdate       # as timedelta

dates = []
for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    dates.append(str(day))

In [3]:
dicc_names = {
    "Caso": ["No. Caso", "ID de caso", "CASO", "id_de_caso"],
    "Tipo": ["Tipo*", "fuente o tipo de contagio", "Fuente_tipo_contagio"],
    "Pais de procedencia" : ["País de procedencia", "Pais de procedencia\t", "Pais a donde viajo  1"],
    "Ciudad": ["Ciudad de ubicación", "Ciudad/Municipio", "Ciudad_municipio", "ciudad_de_ubicaci_n"],
    "Código País": ["Codigo pais"],
    "Ubicación": ["Atención**", "Ubicacion", "atenci_n"],
    "Departamento": ["Departamento o Distrito", "nombre_depa", "departamento"],
    "Fecha_diagnostico": ["Fecha diagnostico", "Fechadiagnostico", "Fecha de diagnóstico", "Fecha de diagnostico"],
    "Fecha_inicio_sintomas": ["Fechadeiniciodesíntomas", "Fecha de inicio de síntomas", "fis"],
    "Tipo_recuperacion": ["Tipo de recuperación", "Tipo_Recuperacion"],
    "Fecha_recuperado": ["Fecha de recuperado", "Fecha recuperado"],
    "Recuperacion": ["Tipo_recuperacion"],
    "Edad": ["edad"],
    "Sexo": ["sexo"]
}
cols2drop = [
    'Unnamed: 1',
    'Pais_viajo_1_cod',
    'Pais_viajo_1_nom',
    'Estado.1',
    "Pais_viajo_1"
]

In [4]:
def rename_cols(df):
    if "Departamento" in df.columns:
        if df["Departamento"].dtype.str == "<i8":
            df = df.drop("Departamento", axis = 1)
    for column in df.columns:
        for name in dicc_names:
            if column in dicc_names[name]:
                df = df.rename(columns={column:name})
    return df

In [5]:
def load_colombia_df():
    client = Socrata("www.datos.gov.co", None)  # https://www.datos.gov.co/es/profile/edit/developer_settings   por si no funciona
    results = client.get("gt2j-8ykr", limit=1000000)
    results_df = pd.DataFrame.from_records(results)
    return results_df

In [36]:
df = load_colombia_df()

In [39]:
df.columns

Index(['id_de_caso', 'fecha_de_notificaci_n', 'c_digo_divipola',
       'ciudad_de_ubicaci_n', 'departamento', 'atenci_n', 'edad', 'sexo',
       'tipo', 'estado', 'pa_s_de_procedencia', 'fis', 'fecha_diagnostico',
       'fecha_recuperado', 'fecha_reporte_web', 'tipo_recuperaci_n',
       'codigo_departamento', 'codigo_pais', 'pertenencia_etnica',
       'nombre_grupo_etnico', 'fecha_de_muerte'],
      dtype='object')

In [41]:
df.loc[:,["id_de_caso", "atenci_n"]].to_csv("a.csv")

In [37]:
df.to_csv("a.csv")

In [12]:
muertes = df[df["fecha_de_muerte"].notnull()]["id_de_caso"].values

In [19]:
big_df = pd.DataFrame()

for date in tqdm(dates):
    try:
        link =f'https://www.ins.gov.co/BoletinesCasosCOVID19Colombia/{date}.xlsx'
        data = pd.read_excel(link)
        data["fecha_boletin"] = date
        data_aux = data.copy(deep=True)
        data = rename_cols(data)
        data = data.drop(cols2drop, axis=1, errors="ignore")
        data = data[data["Caso"].isin(muertes)]
        big_df = big_df.append(data)
    except Exception as e:
        print("ended in " + date)
        print(f"error: {e}")
        continue
    



# big_df = big_df.loc[:,["fecha_boletin", "Caso", "Departamento", "Ciudad", "Edad", "Sexo", "Ubicación", "Estado", "Código ciudad"]]

 14%|█▍        | 15/107 [00:11<01:21,  1.13it/s]

ended in 2020-04-11
error: 'Caso'


 97%|█████████▋| 104/107 [20:09<01:15, 25.03s/it]

ended in 2020-07-09
error: HTTP Error 404: NOT FOUND


 98%|█████████▊| 105/107 [20:09<00:35, 17.62s/it]

ended in 2020-07-10
error: HTTP Error 404: NOT FOUND


 99%|█████████▉| 106/107 [20:10<00:12, 12.46s/it]

ended in 2020-07-11
error: HTTP Error 404: NOT FOUND


100%|██████████| 107/107 [20:10<00:00, 11.32s/it]

ended in 2020-07-12
error: HTTP Error 404: NOT FOUND


In [31]:
big_df["Ubicación"] = big_df["Ubicación"].str.lower().str.strip()

In [34]:
big_df["Ubicación"].value_counts(normalize=True)

fallecido                0.789819
hospital                 0.091466
casa                     0.065933
hospital uci             0.043587
recuperado               0.008924
en casa                  0.000228
recuperado (hospital)    0.000043
Name: Ubicación, dtype: float64

In [42]:
bd = big_df.loc[:,["Caso", "Ubicación"]]
bd = bd[bd["Ubicación"]!="fallecido"]
bd.groupby("Caso").tail(1)["Ubicación"].value_counts(normalize=True)*100

hospital                 44.283778
casa                     32.806146
hospital uci             19.882512
recuperado                2.959783
en casa                   0.045188
recuperado (hospital)     0.022594
Name: Ubicación, dtype: float64

In [17]:
big_df.to_csv("julio11.csv", index=False)